In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import datetime
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from scipy.optimize import curve_fit
import glob
import os
import shutil
import plotly.graph_objects as go
from openpyxl import load_workbook
import math

def dp_to_rh (dp,T):
     return  100 * (math.e**(17.625 * dp/(243.04 + dp))/math.e**(17.625 * T/(243.04 + T)) )


from scipy import constants
flow_rate=2475.8/60 # mL s-1          #measured 20231005
volume_solution=1*600*1e-6    # mass in 0.025 L total solution;  taking 600uL from this solution
T=273+18
pressure=949 #mbar
molecules= 2.46*1e10*298/T*pressure/1013.25 #  molecules/(cm3) per one ppb
surface_area=np.pi*1*43.8 #m2 diameter=1 cm length = 43.8 cm this is the lamp length.  the cwft tube is about 5 cm longer.

factor=3/4.58  #to normalized light intensity
flux_constant=molecules*flow_rate/surface_area  # ppb multiply this constant will get flux (molecules cm-2 s-1)
j_constant=1*molecules/constants.N_A*flow_rate*factor/volume_solution   # ppb/M_NaNO3 multiply this constant will get j  (s-1)

In [65]:
# nox_path='data/T200_NOx_/2023-10-05_09.11.18_PC14751_T200_NOx.dat'
# nox_path='data/T200_NOx_/2023-10-05_09.11.18_PC14751_T200_NOx.dat'
nox_path='data/T200_NOx_/2023-10-11_09.18.25_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_10_11_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_10_11_Channel1_Data.dat'
# rh_path='data/Humidity_Sensors_/2023-10-05_09.11.20_PC14751_Humidity_Sensors.dat'
rh_path='data/Humidity_Sensors_/2023-10-11_09.18.24_PC14751_Humidity_Sensors.dat'

hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 


df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']
df_no2_icad['no2']=no2['NO2 (ppb)']
df_nox_t200['no']=nox['NO']
df_nox_t200['NO2']=nox['NO2']

average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index
df_rh['rh']=dp_to_rh (df_rh.dew_point,18)
df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-10-05 8:30:00'])

# df.hono=df.hono-df.hono[46:76].mean()
time_start=0
# df=df[time_start:time_start+30+30]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=df_hono_icad.date_time[400:1060],y=df_hono_icad.hono[400:1060],yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x=df_no2_icad.date_time[400:1060],y=df_no2_icad.no2[400:1060],yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO2"))
fig_1.add_trace(go.Scatter(x= df_nox_t200.date_time[:600],y=df_nox_t200.no[:600],yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))
fig_1.add_trace(go.Scatter(x= df_rh.date_time[:],y=df_rh.rh[:],yaxis = "y2",mode='markers+lines',marker_size=10,line_width=2,name="RH"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
name='1011total'
fig_1.update_layout(title=name+' NH4NO3 (0.1M) + DHB (0.02M)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1500, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', zeroline=False,
               range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.65 ,xanchor="left",x=0.85,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

# fig_1.add_vrect(x0=10, x1=40,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
fig_1.write_html(name+'.html')
fig_1.write_image(name+'.png')

Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh]
Index: []


In [66]:
# nox_path='data/T200_NOx_/2023-10-05_09.11.18_PC14751_T200_NOx.dat'
# nox_path='data/T200_NOx_/2023-10-05_09.11.18_PC14751_T200_NOx.dat'
nox_path='data/T200_NOx_/2023-10-11_09.18.25_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_10_11_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_10_11_Channel1_Data.dat'
# rh_path='data/Humidity_Sensors_/2023-10-05_09.11.20_PC14751_Humidity_Sensors.dat'
rh_path='data/Humidity_Sensors_/2023-10-11_09.18.24_PC14751_Humidity_Sensors.dat'

hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 


df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']
df_no2_icad['no2']=no2['NO2 (ppb)']
df_nox_t200['no']=nox['NO']
df_nox_t200['NO2']=nox['NO2']

average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index
df_rh['rh']=dp_to_rh (df_rh.dew_point,18)
df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-10-05 8:30:00'])

# df.hono=df.hono-df.hono[46:76].mean()

name='1011d'
time_start=218-10
df=df[time_start:time_start+30+20]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.rh,yaxis = "y2",mode='markers+lines',marker_size=10,line_width=2,name="RH"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title=name+' NH4NO3 (1M) + DHB (0.01M)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', zeroline=False,
              #  range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),range=[-2,68],tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=10, x1=30,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
fig_1.write_html(name+'.html')
fig_1.write_image(name+'.png')

Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh]
Index: []


In [67]:
print(df[df['date_time'] == '2023-10-11 12:56:00'])


        hono           date_time      no2      no     NO2  dew_point       rh  \
218 -0.03600 2023-10-11 12:56:00 -0.01998 0.20000 0.04737   15.10000 83.16449   

     x_minutes  
218         10  


In [68]:
from scipy import constants
# flow_rate=2475.8/60 # mL s-1         
mol_NaNO3=1*600*1e-6    # mass in 0.025 L total solution;  taking 600uL from this solution
T=273+18
molecules= 2.46*1e10*273/T #  molecules/(cm3) per one ppb
# mean_ppb=   # average ppb in 30 min
# mol_per_s=mean_ppb*molecules/constants.N_A*flow_rate   # mol/s
# photolysis_rate_constant=mol_per_s/mol_NaNO3  #s-1
# photolysis_rate_constant

time = 30 # min
photolysis_rate_constant=np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3
production_amount=np.trapz(df.hono[10:40])*molecules*flow_rate*60/constants.N_A   # total mol in the irradiation time

# photolysis_rate_constant
j_pd=pd.DataFrame()
j_pd['nitrate_M']=[1]

j_pd['hono']=[np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no']=[np.trapz(df.no[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no2']=[np.trapz(df.no2[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['total']=[np.trapz(((df.no+df.no2+df.hono))[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]

data_compile=pd.DataFrame(np.zeros((1, 5)))
data_compile.columns=j_pd.columns
data_compile.iloc[0]=j_pd.iloc[0]
data_compile

,nitrate_M,hono,no,no2,total
0,1.00000,0.00000,0.00000,0.00000,0.00000


In [69]:
# nox_path='data/T200_NOx_/2023-10-05_09.11.18_PC14751_T200_NOx.dat'
# nox_path='data/T200_NOx_/2023-10-05_09.11.18_PC14751_T200_NOx.dat'
nox_path='data/T200_NOx_/2023-10-11_09.18.25_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_10_11_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_10_11_Channel1_Data.dat'
# rh_path='data/Humidity_Sensors_/2023-10-05_09.11.20_PC14751_Humidity_Sensors.dat'
rh_path='data/Humidity_Sensors_/2023-10-11_09.18.24_PC14751_Humidity_Sensors.dat'

hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 


df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']
df_no2_icad['no2']=no2['NO2 (ppb)']
df_nox_t200['no']=nox['NO']
df_nox_t200['NO2']=nox['NO2']

average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index
df_rh['rh']=dp_to_rh (df_rh.dew_point,18)
df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-10-05 8:30:00'])

# df.hono=df.hono-df.hono[46:76].mean()

name='1011e'
time_start=258-10
df=df[time_start:time_start+30+20]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.rh,yaxis = "y2",mode='markers+lines',marker_size=10,line_width=2,name="RH"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title=name+' NH4NO3 (0.5 M) + DHB (0.01M)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', zeroline=False,
              #  range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),range=[-2,68],tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=10, x1=30,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
fig_1.write_html(name+'.html')
fig_1.write_image(name+'.png')

Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh]
Index: []


In [70]:
from scipy import constants
# flow_rate=2475.8/60 # mL s-1         
mol_NaNO3=0.5*600*1e-6    # mass in 0.025 L total solution;  taking 600uL from this solution
T=273+18
molecules= 2.46*1e10*273/T #  molecules/(cm3) per one ppb
# mean_ppb=   # average ppb in 30 min
# mol_per_s=mean_ppb*molecules/constants.N_A*flow_rate   # mol/s
# photolysis_rate_constant=mol_per_s/mol_NaNO3  #s-1
# photolysis_rate_constant

time = 30 # min
photolysis_rate_constant=np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3
production_amount=np.trapz(df.hono[10:40])*molecules*flow_rate*60/constants.N_A   # total mol in the irradiation time

# photolysis_rate_constant
j_pd=pd.DataFrame()
j_pd['nitrate_M']=[0.5]

j_pd['hono']=[np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no']=[np.trapz(df.no[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no2']=[np.trapz(df.no2[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['total']=[np.trapz(((df.no+df.no2+df.hono))[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]

data_compile=data_compile.append(j_pd)

/var/folders/79/t5bzbbwx1pn2dyslxyqgpk940000gn/T/ipykernel_70469/678275986.py:24: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [71]:
print(df[df['date_time'] == '2023-10-11 13:36:00']) #e 0.5M

print(df[df['date_time'] == '2023-10-11 14:16:00']) #f0.2M

print(df[df['date_time'] == '2023-10-11 14:53:00']) #g 0.1 M



       hono           date_time      no2      no      NO2  dew_point       rh  \
258 0.00150 2023-10-11 13:36:00 -0.00710 0.12000 -0.06500   14.63448 80.70831   

     x_minutes  
258         10  
Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh, x_minutes]
Index: []
Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh, x_minutes]
Index: []


In [72]:
# nox_path='data/T200_NOx_/2023-10-05_09.11.18_PC14751_T200_NOx.dat'
# nox_path='data/T200_NOx_/2023-10-05_09.11.18_PC14751_T200_NOx.dat'
nox_path='data/T200_NOx_/2023-10-11_09.18.25_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_10_11_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_10_11_Channel1_Data.dat'
# rh_path='data/Humidity_Sensors_/2023-10-05_09.11.20_PC14751_Humidity_Sensors.dat'
rh_path='data/Humidity_Sensors_/2023-10-11_09.18.24_PC14751_Humidity_Sensors.dat'

hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 


df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']
df_no2_icad['no2']=no2['NO2 (ppb)']
df_nox_t200['no']=nox['NO']
df_nox_t200['NO2']=nox['NO2']

average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index
df_rh['rh']=dp_to_rh (df_rh.dew_point,18)
df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-10-05 8:30:00'])

# df.hono=df.hono-df.hono[46:76].mean()

name='1011f'
time_start=302-10
df=df[time_start:time_start+30+10]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x[5:],y=df.no[5:],yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.rh,yaxis = "y2",mode='markers+lines',marker_size=10,line_width=2,name="RH"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title=name+' NH4NO3 (0.2 M) + DHB (0.01M)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', zeroline=False,
              #  range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),range=[-2,68],tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=7, x1=27,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
fig_1.write_html(name+'.html')
fig_1.write_image(name+'.png')

Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh]
Index: []


In [73]:
from scipy import constants
# flow_rate=2475.8/60 # mL s-1         
mol_NaNO3=0.2*600*1e-6    # mass in 0.025 L total solution;  taking 600uL from this solution
T=273+18
molecules= 2.46*1e10*273/T #  molecules/(cm3) per one ppb
# mean_ppb=   # average ppb in 30 min
# mol_per_s=mean_ppb*molecules/constants.N_A*flow_rate   # mol/s
# photolysis_rate_constant=mol_per_s/mol_NaNO3  #s-1
# photolysis_rate_constant

time = 30 # min
photolysis_rate_constant=np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3
production_amount=np.trapz(df.hono[10:40])*molecules*flow_rate*60/constants.N_A   # total mol in the irradiation time

# photolysis_rate_constant
j_pd=pd.DataFrame()
j_pd['nitrate_M']=[0.2]
j_pd['hono']=[np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no']=[np.trapz(df.no[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no2']=[np.trapz(df.no2[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['total']=[np.trapz(((df.no+df.no2+df.hono))[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]

data_compile=data_compile.append(j_pd)


/var/folders/79/t5bzbbwx1pn2dyslxyqgpk940000gn/T/ipykernel_70469/1976548334.py:23: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [74]:
# nox_path='data/T200_NOx_/2023-10-05_09.11.18_PC14751_T200_NOx.dat'
# nox_path='data/T200_NOx_/2023-10-05_09.11.18_PC14751_T200_NOx.dat'
nox_path='data/T200_NOx_/2023-10-11_09.18.25_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_10_11_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_10_11_Channel1_Data.dat'
# rh_path='data/Humidity_Sensors_/2023-10-05_09.11.20_PC14751_Humidity_Sensors.dat'
rh_path='data/Humidity_Sensors_/2023-10-11_09.18.24_PC14751_Humidity_Sensors.dat'

hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 


df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']
df_no2_icad['no2']=no2['NO2 (ppb)']
df_nox_t200['no']=nox['NO']
df_nox_t200['NO2']=nox['NO2']

average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index
df_rh['rh']=dp_to_rh (df_rh.dew_point,18)
df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-10-05 8:30:00'])

# df.hono=df.hono-df.hono[46:76].mean()

name='1011g'
time_start=335-10
df=df[time_start:time_start+30+20]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.rh,yaxis = "y2",mode='markers+lines',marker_size=10,line_width=2,name="RH"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))

fig_1.update_layout(title=name+' NH4NO3 (0.1 M) + DHB (0.01M)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', zeroline=False,
              #  range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),range=[-2,68],tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=10, x1=30,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
fig_1.write_html(name+'.html')
fig_1.write_image(name+'.png')

Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh]
Index: []


In [75]:
from scipy import constants
# flow_rate=2475.8/60 # mL s-1         
mol_NaNO3=0.1*600*1e-6    # mass in 0.025 L total solution;  taking 600uL from this solution
T=273+18
molecules= 2.46*1e10*273/T #  molecules/(cm3) per one ppb
# mean_ppb=   # average ppb in 30 min
# mol_per_s=mean_ppb*molecules/constants.N_A*flow_rate   # mol/s
# photolysis_rate_constant=mol_per_s/mol_NaNO3  #s-1
# photolysis_rate_constant

time = 30 # min
photolysis_rate_constant=np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3
production_amount=np.trapz(df.hono[10:40])*molecules*flow_rate*60/constants.N_A   # total mol in the irradiation time

# photolysis_rate_constant
j_pd=pd.DataFrame()
j_pd['nitrate_M']=[0.1]
# data_compile.append(j_pd)
j_pd['hono']=[np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no']=[np.trapz(df.no[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no2']=[np.trapz(df.no2[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['total']=[np.trapz(((df.no+df.no2+df.hono))[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
data_compile=data_compile.append(j_pd)



/var/folders/79/t5bzbbwx1pn2dyslxyqgpk940000gn/T/ipykernel_70469/976894795.py:23: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [76]:
nox_path='data/T200_NOx_/2023-10-10_08.15.10_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_10_10_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_10_10_Channel1_Data.dat'
# rh_path='data/Humidity_Sensors_/2023-10-05_09.11.20_PC14751_Humidity_Sensors.dat'
rh_path='data/Humidity_Sensors_/2023-10-10_08.15.07_PC14751_Humidity_Sensors.dat'
hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 


df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']
df_no2_icad['no2']=no2['NO2 (ppb)']
df_nox_t200['no']=nox['NO']
df_nox_t200['NO2']=nox['NO2']

average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index
df_rh['rh']=dp_to_rh (df_rh.dew_point,18)
df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-10-05 8:30:00'])


name='1010d'
time_start=219-10
df=df[time_start:time_start+30+20]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.rh,yaxis = "y2",mode='markers+lines',marker_size=10,line_width=2,name="RH"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))

fig_1.update_layout(title=name+' NH4NO3 (0.01 M) + DHB (0.01M)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', zeroline=False,
              #  range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),range=[-2,68],tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=10, x1=30,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
fig_1.write_html(name+'.html')
fig_1.write_image(name+'.png')

Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh]
Index: []


In [77]:
from scipy import constants
# flow_rate=2475.8/60 # mL s-1         
mol_NaNO3=0.1*600*1e-6    # mass in 0.025 L total solution;  taking 600uL from this solution
T=273+18
molecules= 2.46*1e10*273/T #  molecules/(cm3) per one ppb
# mean_ppb=   # average ppb in 30 min
# mol_per_s=mean_ppb*molecules/constants.N_A*flow_rate   # mol/s
# photolysis_rate_constant=mol_per_s/mol_NaNO3  #s-1
# photolysis_rate_constant

time = 30 # min
photolysis_rate_constant=np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3
production_amount=np.trapz(df.hono[10:40])*molecules*flow_rate*60/constants.N_A   # total mol in the irradiation time

# photolysis_rate_constant
j_pd=pd.DataFrame()
j_pd['nitrate_M']=[0.2]
# data_compile.append(j_pd)
j_pd['hono']=[np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no']=[np.trapz(df.no[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no2']=[np.trapz(df.no2[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['total']=[np.trapz(((df.no+df.no2+df.hono))[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]

data_compile=data_compile.append(j_pd)


/var/folders/79/t5bzbbwx1pn2dyslxyqgpk940000gn/T/ipykernel_70469/348527225.py:24: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [78]:
nox_path='data/T200_NOx_/2023-10-10_08.15.10_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_10_10_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_10_10_Channel1_Data.dat'
# rh_path='data/Humidity_Sensors_/2023-10-05_09.11.20_PC14751_Humidity_Sensors.dat'
rh_path='data/Humidity_Sensors_/2023-10-10_08.15.07_PC14751_Humidity_Sensors.dat'
hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 


df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']
df_no2_icad['no2']=no2['NO2 (ppb)']
df_nox_t200['no']=nox['NO']
df_nox_t200['NO2']=nox['NO2']

average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index
df_rh['rh']=dp_to_rh (df_rh.dew_point,18)
df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-10-05 8:30:00'])


name='1010e'
time_start=363-10
df=df[time_start:time_start+30+20]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.rh,yaxis = "y2",mode='markers+lines',marker_size=10,line_width=2,name="RH"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))

fig_1.update_layout(title=name+' NH4NO3 (0.01 M) + DHB (0.01M)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', zeroline=False,
              #  range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),range=[-2,68],tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

fig_1.add_vrect(x0=10, x1=30,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
fig_1.write_html(name+'.html')
fig_1.write_image(name+'.png')

from scipy import constants
j_pd=pd.DataFrame()
j_pd['nitrate_M']=[0.01]
# flow_rate=2475.8/60 # mL s-1         
mol_NaNO3=j_pd['nitrate_M'][0]*600*1e-6    # mass in 0.025 L total solution;  taking 600uL from this solution
T=273+18
molecules= 2.46*1e10*273/T #  molecules/(cm3) per one ppb

time = 30 # min
photolysis_rate_constant=np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3
production_amount=np.trapz(df.hono[10:40])*molecules*flow_rate*60/constants.N_A   # total mol in the irradiation time

# photolysis_rate_constant

# data_compile.append(j_pd)
j_pd['hono']=[np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no']=[np.trapz(df.no[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no2']=[np.trapz(df.no2[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['total']=[np.trapz(((df.no+df.no2+df.hono))[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]

data_compile=data_compile.append(j_pd)


Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh]
Index: []


/var/folders/79/t5bzbbwx1pn2dyslxyqgpk940000gn/T/ipykernel_70469/2865143338.py:128: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [79]:
nox_path='data/T200_NOx_/2023-10-10_08.15.10_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_10_10_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_10_10_Channel1_Data.dat'
# rh_path='data/Humidity_Sensors_/2023-10-05_09.11.20_PC14751_Humidity_Sensors.dat'
rh_path='data/Humidity_Sensors_/2023-10-10_08.15.07_PC14751_Humidity_Sensors.dat'
hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 


df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']
df_no2_icad['no2']=no2['NO2 (ppb)']
df_nox_t200['no']=nox['NO']
df_nox_t200['NO2']=nox['NO2']

average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index
df_rh['rh']=dp_to_rh (df_rh.dew_point,18)
df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-10-05 8:30:00'])


name='1010f'
time_start=408-10
df=df[time_start:time_start+30+20]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
# x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.rh,yaxis = "y2",mode='markers+lines',marker_size=10,line_width=2,name="RH"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))

fig_1.update_layout(title=name+' NH4NO3 (0.05 M) + DHB (0.01M)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', zeroline=False,
              #  range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),range=[-2,68],tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.8 ,xanchor="left",x=0.05,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
fig_1.add_vrect(x0=10, x1=30,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
fig_1.write_html(name+'.html')
fig_1.write_image(name+'.png')


from scipy import constants
j_pd=pd.DataFrame()
j_pd['nitrate_M']=[0.05]
# flow_rate=2475.8/60 # mL s-1         
mol_NaNO3=j_pd['nitrate_M'][0]*600*1e-6    # mass in 0.025 L total solution;  taking 600uL from this solution
T=273+18
molecules= 2.46*1e10*273/T #  molecules/(cm3) per one ppb

time = 30 # min
photolysis_rate_constant=np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3
production_amount=np.trapz(df.hono[10:40])*molecules*flow_rate*60/constants.N_A   # total mol in the irradiation time

# photolysis_rate_constant

# data_compile.append(j_pd)
j_pd['hono']=[np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no']=[np.trapz(df.no[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no2']=[np.trapz(df.no2[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['total']=[np.trapz(((df.no+df.no2+df.hono))[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]

data_compile=data_compile.append(j_pd)


Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh]
Index: []


/var/folders/79/t5bzbbwx1pn2dyslxyqgpk940000gn/T/ipykernel_70469/2046548550.py:128: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [80]:
nox_path='data/T200_NOx_/2023-10-10_08.15.10_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_10_10_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_10_10_Channel1_Data.dat'
# rh_path='data/Humidity_Sensors_/2023-10-05_09.11.20_PC14751_Humidity_Sensors.dat'
rh_path='data/Humidity_Sensors_/2023-10-10_08.15.07_PC14751_Humidity_Sensors.dat'
hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 


df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']
df_no2_icad['no2']=no2['NO2 (ppb)']
df_nox_t200['no']=nox['NO']
df_nox_t200['NO2']=nox['NO2']

average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index
df_rh['rh']=dp_to_rh (df_rh.dew_point,18)
df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-10-05 8:30:00'])


name='1010g'
time_start=504-10
df=df[time_start:time_start+30+30]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.rh,yaxis = "y2",mode='markers+lines',marker_size=10,line_width=2,name="RH"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title=name+' NH4NO3 (0.1M) + DHB (0.02M)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1500, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', zeroline=False,
               range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.65 ,xanchor="left",x=0.85,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

# fig_1.add_vrect(x0=10, x1=40,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
fig_1.write_html(name+'.html')
fig_1.write_image(name+'.png')


from scipy import constants
j_pd=pd.DataFrame()
j_pd['nitrate_M']=[0.2]
# flow_rate=2475.8/60 # mL s-1         
mol_NaNO3=j_pd['nitrate_M'][0]*600*1e-6    # mass in 0.025 L total solution;  taking 600uL from this solution
T=273+18
molecules= 2.46*1e10*273/T #  molecules/(cm3) per one ppb

time = 30 # min
photolysis_rate_constant=np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3
production_amount=np.trapz(df.hono[10:40])*molecules*flow_rate*60/constants.N_A   # total mol in the irradiation time

# photolysis_rate_constant

# data_compile.append(j_pd)
j_pd['hono']=[np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no']=[np.trapz(df.no[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no2']=[np.trapz(df.no2[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['total']=[np.trapz(((df.no+df.no2+df.hono))[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]

data_compile=data_compile.append(j_pd)


Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh]
Index: []


/var/folders/79/t5bzbbwx1pn2dyslxyqgpk940000gn/T/ipykernel_70469/1205424714.py:128: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [81]:
nox_path='data/T200_NOx_/2023-10-10_08.15.10_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_10_10_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_10_10_Channel1_Data.dat'
# rh_path='data/Humidity_Sensors_/2023-10-05_09.11.20_PC14751_Humidity_Sensors.dat'
rh_path='data/Humidity_Sensors_/2023-10-10_08.15.07_PC14751_Humidity_Sensors.dat'
hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 


df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']
df_no2_icad['no2']=no2['NO2 (ppb)']
df_nox_t200['no']=nox['NO']
df_nox_t200['NO2']=nox['NO2']

average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index
df_rh['rh']=dp_to_rh (df_rh.dew_point,18)
df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-10-05 8:30:00'])


name='1010h'
time_start=544-10
df=df[time_start:time_start+30+30]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.rh,yaxis = "y2",mode='markers+lines',marker_size=10,line_width=2,name="RH"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title=name+' NH4NO3 (0.1M) + DHB (0.02M)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1500, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', zeroline=False,
               range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.65 ,xanchor="left",x=0.85,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

# fig_1.add_vrect(x0=10, x1=40,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
fig_1.write_html(name+'.html')
fig_1.write_image(name+'.png')


from scipy import constants
j_pd=pd.DataFrame()
j_pd['nitrate_M']=[0.05]
# flow_rate=2475.8/60 # mL s-1         
mol_NaNO3=j_pd['nitrate_M'][0]*600*1e-6    # mass in 0.025 L total solution;  taking 600uL from this solution
T=273+18
molecules= 2.46*1e10*273/T #  molecules/(cm3) per one ppb

time = 30 # min
photolysis_rate_constant=np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3
production_amount=np.trapz(df.hono[10:40])*molecules*flow_rate*60/constants.N_A   # total mol in the irradiation time

# photolysis_rate_constant

# data_compile.append(j_pd)
j_pd['hono']=[np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no']=[np.trapz(df.no[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no2']=[np.trapz(df.no2[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['total']=[np.trapz(((df.no+df.no2+df.hono))[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]

data_compile=data_compile.append(j_pd)


Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh]
Index: []


/var/folders/79/t5bzbbwx1pn2dyslxyqgpk940000gn/T/ipykernel_70469/1599814108.py:128: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [82]:
nox_path='data/T200_NOx_/2023-10-10_08.15.10_PC14751_T200_NOx.dat'
no2_path='data/NO2_ICAD/2023_10_10_Channel1_Data.dat'
hono_path='data/HONO_ICAD/2023_10_10_Channel1_Data.dat'
# rh_path='data/Humidity_Sensors_/2023-10-05_09.11.20_PC14751_Humidity_Sensors.dat'
rh_path='data/Humidity_Sensors_/2023-10-10_08.15.07_PC14751_Humidity_Sensors.dat'
hono=pd.read_csv(hono_path,sep = '\t', engine='python') #HONO from ICAD
no2=pd.read_csv(no2_path,sep = '\t', engine='python') #NO2 from ICAD
nox=pd.read_csv(nox_path,sep = '\t', header=1, engine='python') #no of nox analyzer
rh=pd.read_csv(rh_path,sep = '\t', header=1, encoding= 'unicode_escape', engine='python') #RH data

#ICAD
hono['Start Date/Time (UTC)']=pd.to_datetime(hono['Start Date/Time (UTC)'])
no2['Start Date/Time (UTC)']=pd.to_datetime(no2['Start Date/Time (UTC)'])

#NOx analyzer
nox['datetime']=nox['Date']+ nox['Time']
nox['time']= pd.to_datetime(nox['datetime'], format='%d.%m.%Y%H:%M:%S') 


#RH sensor
rh['datetime']=rh['Date']+ rh['Time']
rh['time']= pd.to_datetime(rh['datetime'], format='%d.%m.%Y%H:%M:%S') 

hono['Time_correct']=hono['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
hono['Time_correct']= pd.to_datetime(hono['Time_correct'], format='%d.%m.%Y%H:%M:%S') 

no2['Time_correct']=no2['Start Date/Time (UTC)']+timedelta(hours=2, minutes=1)
no2['Time_correct']= pd.to_datetime(no2['Time_correct'], format='%d.%m.%Y%H:%M:%S') 


df_hono_icad=pd.DataFrame()
df_no2_icad=pd.DataFrame()
df_rh=pd.DataFrame()
df_nox_t200=pd.DataFrame()

df_hono_icad['time']=hono['Time_correct']
df_no2_icad['time']=no2['Time_correct']
df_nox_t200['time']=nox['time']
df_rh['time']=rh['time']
df_rh['dew_point']=rh['DP7[°C]']
df_hono_icad['hono']=hono['HONO (ppb)']
df_no2_icad['no2']=no2['NO2 (ppb)']
df_nox_t200['no']=nox['NO']
df_nox_t200['NO2']=nox['NO2']

average_interval='60s'
df_hono_icad=df_hono_icad.resample(average_interval,on='time').mean()
df_hono_icad['date_time']=df_hono_icad.index

df_no2_icad=df_no2_icad.resample(average_interval,on='time').mean()
df_no2_icad['date_time']=df_no2_icad.index
df_no2_icad['date_time']= pd.to_datetime(df_no2_icad['date_time']) 

df_rh=df_rh.resample(average_interval,on='time').mean()
df_rh['date_time']=df_rh.index
df_rh['rh']=dp_to_rh (df_rh.dew_point,18)
df_nox_t200=df_nox_t200.resample(average_interval,on='time').mean()
df_nox_t200['date_time']=df_nox_t200.index

df=pd.merge(pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time'),df_rh,on='date_time')
# df=pd.merge(pd.merge(df_hono_icad,df_no2_icad,on='date_time'),df_nox_t200,on='date_time')
# df=pd.merge(df_hono_icad,df_no2_icad,on='date_time')

# correct HONO background
print(df[df['date_time'] == '2023-10-05 8:30:00'])


name='1010i'
time_start=598-10
df=df[time_start:time_start+30+30]
# df=df[time_start:time_start+39]

x_minutes=(df['date_time'] - df['date_time'][time_start]).dt.total_seconds().div(60).astype(int)
df['x_minutes']=x_minutes

x= df.x_minutes
x= df.date_time

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=x,y=df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="HONO"))
fig_1.add_trace(go.Scatter(x= x,y=df.no2,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x= x,y=df.no,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="NO"))

# fig_1.add_trace(go.Scatter(x= x,y=df.rh,yaxis = "y2",mode='markers+lines',marker_size=10,line_width=2,name="RH"))
# fig_1.add_trace(go.Scatter(x= df.date_time,y=df.no+df.no2+df.hono,yaxis = "y1",mode='markers+lines',marker_size=10,line_width=2,name="no"))
fig_1.update_layout(title=name+' NH4NO3 (0.1M) + DHB (0.02M)',titlefont=dict(color="black",size=25))
# fig_1.update_layout( bargap=0)
fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1500, height=800,
    xaxis=dict( title="time (min)",titlefont=dict(color="black",size=25),showline=True, mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis=dict( title='HONO or NO<sub>x</sub> concentration (ppb)', zeroline=False,
               range=[-2,68],
               titlefont=dict(color="black",size=25),mirror= True,tickfont=dict(color='black',size=25 ),linecolor="black", linewidth=2,ticks='inside'),
    yaxis2=dict(title='',titlefont=dict(color="black",size=25),tickfont=dict(color="black" ,size=25),ticks='inside',anchor="free",overlaying="y", side="right", position=1, linecolor="black", linewidth=2,showticklabels = True) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.65 ,xanchor="left",x=0.85,  font=dict( size=25,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)

# fig_1.add_vrect(x0=10, x1=40,annotation_text="UV-A lights on", annotation_position="top",annotation_font_size=14,fillcolor="yellow", opacity=0.3,layer="below", line_width=0)

config = {'displayModeBar': True}
fig_1.show(config=config)
fig_1.write_html(name+'.html')
fig_1.write_image(name+'.png')


from scipy import constants
j_pd=pd.DataFrame()
j_pd['nitrate_M']=[0.07]
flow_rate=2475.8/60 # mL s-1         
mol_NaNO3=j_pd['nitrate_M'][0]*600*1e-6    # mass in 0.025 L total solution;  taking 600uL from this solution
T=273+18
molecules= 2.46*1e10*273/T #  molecules/(cm3) per one ppb

time = 30 # min
photolysis_rate_constant=np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3
production_amount=np.trapz(df.hono[10:40])*molecules*flow_rate*60/constants.N_A   # total mol in the irradiation time

# photolysis_rate_constant

# data_compile.append(j_pd)
j_pd['hono']=[np.trapz(df.hono[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no']=[np.trapz(df.no[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['no2']=[np.trapz(df.no2[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]
j_pd['total']=[np.trapz(((df.no+df.no2+df.hono))[10:40])/time*molecules/constants.N_A*flow_rate /mol_NaNO3]

data_compile=data_compile.append(j_pd)


Empty DataFrame
Columns: [hono, date_time, no2, no, NO2, dew_point, rh]
Index: []


/var/folders/79/t5bzbbwx1pn2dyslxyqgpk940000gn/T/ipykernel_70469/3589527447.py:128: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [83]:
data_compile=data_compile.sort_values('nitrate_M')

In [84]:
data_compile["DHB_M"]=0.01
data_compile["DHB_nitrate_ratio"]=data_compile["nitrate_M"]/0.01
data_compile["nitrate_molecules_density"]=data_compile["nitrate_M"]*volume_solution*constants.N_A/surface_area

In [85]:
# data_compile.index=[1,0.5,0.2,0.1]

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=data_compile.DHB_nitrate_ratio,y=data_compile.hono*1e6/4.85*3,yaxis = "y1",mode='markers',marker_size=20,line_width=3,name="HONO"))
fig_1.add_trace(go.Scatter(x=data_compile.DHB_nitrate_ratio,y=data_compile.no2*1e6/4.85*3,yaxis = "y1",mode='markers',marker_size=20,line_width=3,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x=data_compile.DHB_nitrate_ratio,y=9E-08*data_compile.nitrate_M**(-0.506)*1e6/4.85*3,yaxis = "y1",line_color='black',mode='lines',marker_size=20,
                           line_width=3,name="Total(fit)"))
name='Nitrate_dependence'


fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="Ratio of NH<sub>4</sub>NO<sub>3</sub> to DHB",titlefont=dict(color="black",size=40),ticklen=6,tickwidth=3,  
               showline=True, mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside'),
    yaxis=dict( title='Normalzied photolysis rate constant (10<sup>-6</sup> s<sup>-1</sup>)', zeroline=False,
              #  range=[0,1.5],
               titlefont=dict(color="black",size=30),mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside',ticklen=6,tickwidth=3,
               ),
    yaxis2=dict(title='',titlefont=dict(color="black",size=40),tickfont=dict(color="black" ,size=40),ticks='inside',
                anchor="free",overlaying="y", side="right", position=1, ticklen=0,tickwidth=3,linecolor="black", linewidth=2,showticklabels = False) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.95 ,xanchor="left",x=0.7,  font=dict( size=40,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
fig_1.show()
fig_1.write_image('output/'+name+'_gradient.svg')


In [86]:
flux_df = {}
flux_df["nitrate_M"]=data_compile["nitrate_M"]
flux_df["nitrate_molecules_density"]=data_compile["nitrate_molecules_density"]
flux_df["DHB_M"]=data_compile["DHB_M"]
flux_df["DHB_nitrate_ratio"]=data_compile["DHB_nitrate_ratio"]

for column in ["hono", "no", "no2","total"]:
    flux_df[column] = data_compile[column]*data_compile["nitrate_M"]*volume_solution*constants.N_A/surface_area
flux_df=pd.DataFrame(flux_df)
flux_df
# data_compile.index=[1,0.5,0.2,0.1]

fig_1= go.Figure()
fig_1.add_trace(go.Scatter(x=flux_df.nitrate_molecules_density*1e-20,y=flux_df.hono*1e-11,yaxis = "y1",mode='markers',marker_size=20,line_width=3,name="HONO"))
# fig_1.add_trace(go.Scatter(x=flux_df.nitrate_molecules*1e-20,y=flux_df.no*1e-11,yaxis = "y1",mode='markers',marker_size=20,line_width=3,name="NO"))
# fig_1.add_trace(go.Scatter(x=flux_df.nitrate_molecules*1e-20,y=flux_df.no2*1e-11,yaxis = "y1",mode='markers',marker_size=20,line_width=3,name="NO<sub>2</sub>"))
fig_1.add_trace(go.Scatter(x=flux_df.nitrate_molecules_density*1e-20,y=flux_df.total*1e-11,yaxis = "y1",mode='markers',marker_size=20,line_width=3,name="total"))

name='Nitrate_dependence'

fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="NH<sub>4</sub>NO<sub>3</sub> amount (10<sup>20</sup> molecules cm<sup>-2</sup>)",titlefont=dict(color="black",size=40),ticklen=6,tickwidth=3,  
               showline=True, mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside'),
    yaxis=dict( title='flux (10<sup>11</sup> molecules cm<sup>-2</sup> s<sup>-1</sup>)', zeroline=False,
              #  range=[0,1.5],
               titlefont=dict(color="black",size=30),mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside',ticklen=6,tickwidth=3,
               ),
    yaxis2=dict(title='',titlefont=dict(color="black",size=40),tickfont=dict(color="black" ,size=40),ticks='inside',
                anchor="free",overlaying="y", side="right", position=1, ticklen=0,tickwidth=3,linecolor="black", linewidth=2,showticklabels = False) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.95 ,xanchor="left",x=0.05,  font=dict( size=40,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
fig_1.show()
fig_1.write_image('output/'+name+'_gradient.svg')


In [87]:

flux_df = {}
flux_df["nitrate_M"]=data_compile["nitrate_M"]
flux_df["nitrate_molecules_density"]=data_compile["nitrate_molecules_density"]
flux_df["DHB_M"]=data_compile["DHB_M"]
flux_df["DHB_nitrate_ratio"]=data_compile["DHB_nitrate_ratio"]

for column in ["hono", "no", "no2","total"]:
    flux_df[column] = data_compile[column]*data_compile["nitrate_M"]*volume_solution*constants.N_A/surface_area
flux_df=pd.DataFrame(flux_df)
flux_df
# data_compile.index=[1,0.5,0.2,0.1]

fig_1= go.Figure()
from import_ import linear

x_data=flux_df.nitrate_molecules_density.values
y_data=flux_df.hono.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='red',marker_size=20,line_width=3,name="HONO"))
fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='red',marker_size=20,line_width=3,name="",showlegend=False))



x_data=flux_df.nitrate_molecules_density.values
y_data=flux_df.total.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='blue',marker_size=20,line_width=3,name="total"))
fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='blue',marker_size=20,line_width=3,name="",showlegend=False))

name='Nitrate_dependence'

fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="NH<sub>4</sub>NO<sub>3</sub> amount (10<sup>20</sup> molecules cm<sup>-2</sup>)",titlefont=dict(color="black",size=40),ticklen=6,tickwidth=3,  
               showline=True, mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside'),
    yaxis=dict( title='flux (10<sup>11</sup> molecules cm<sup>-2</sup> s<sup>-1</sup>)', zeroline=False,
              #  range=[0,1.5],
               titlefont=dict(color="black",size=40),mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside',ticklen=6,tickwidth=3,
               ),
    yaxis2=dict(title='',titlefont=dict(color="black",size=40),tickfont=dict(color="black" ,size=40),ticks='inside',
                anchor="free",overlaying="y", side="right", position=1, ticklen=0,tickwidth=3,linecolor="black", linewidth=2,showticklabels = False) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.95 ,xanchor="left",x=0.05,  font=dict( size=40,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
fig_1.show()
fig_1.write_image('output/'+name+'_gradient.svg')

y= 0.0000x + 32638465389.48
R2 = 0.5206
y= 0.0000x + 64128206226.08
R2 = 0.4896


In [88]:

flux_df = {}
flux_df["nitrate_M"]=data_compile["nitrate_M"]
flux_df["nitrate_M"]=data_compile["nitrate_M"]
flux_df["DHB_M"]=data_compile["DHB_M"]
flux_df["DHB_nitrate_ratio"]=data_compile["DHB_nitrate_ratio"]

for column in ["hono", "no", "no2","total"]:
    flux_df[column] = data_compile[column]*data_compile["nitrate_M"]*volume_solution*constants.N_A/surface_area
flux_df=pd.DataFrame(flux_df)
flux_df
# data_compile.index=[1,0.5,0.2,0.1]

fig_1= go.Figure()
from import_ import linear

x_data=flux_df.nitrate_M.values
y_data=flux_df.hono.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data*1e0,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='red',marker_size=20,line_width=3,name="HONO"))
fig_1.add_trace(go.Scatter(x=x_data*1e0,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='red',marker_size=20,line_width=3,name="",showlegend=False))



x_data=flux_df.nitrate_M.values
y_data=flux_df.total.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data*1e0,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='blue',marker_size=20,line_width=3,name="total"))
fig_1.add_trace(go.Scatter(x=x_data*1e0,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='blue',marker_size=20,line_width=3,name="",showlegend=False))

name='Nitrate_dependence_1'

fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="NH<sub>4</sub>NO<sub>3</sub> conc. before equilbirum (M)",titlefont=dict(color="black",size=40),ticklen=6,tickwidth=3,  
               showline=True, mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside'),
    yaxis=dict( title='flux (10<sup>11</sup> molecules cm<sup>-2</sup> s<sup>-1</sup>)', zeroline=False,
              #  range=[0,1.5],
               titlefont=dict(color="black",size=40),mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside',ticklen=6,tickwidth=3,
               ),
    yaxis2=dict(title='',titlefont=dict(color="black",size=40),tickfont=dict(color="black" ,size=40),ticks='inside',
                anchor="free",overlaying="y", side="right", position=1, ticklen=0,tickwidth=3,linecolor="black", linewidth=2,showticklabels = False) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.95 ,xanchor="left",x=0.05,  font=dict( size=40,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
fig_1.show()
fig_1.write_image('output/'+name+'_gradient.svg')

y= 112548313344.0821x + 32638465389.48
R2 = 0.5206
y= 173494679132.2781x + 64128206226.08
R2 = 0.4896


In [89]:

flux_df = {}
flux_df["nitrate_M"]=data_compile["nitrate_M"]
flux_df["nitrate_molecules_density"]=data_compile["nitrate_molecules_density"]
flux_df["DHB_M"]=data_compile["DHB_M"]
flux_df["DHB_nitrate_ratio"]=data_compile["DHB_nitrate_ratio"]

for column in ["hono", "no", "no2","total"]:
    flux_df[column] = data_compile[column]*data_compile["nitrate_M"]*volume_solution*constants.N_A/surface_area
flux_df=pd.DataFrame(flux_df)
flux_df
# data_compile.index=[1,0.5,0.2,0.1]

fig_1= go.Figure()
from import_ import linear

x_data=flux_df.nitrate_molecules_density.values
y_data=flux_df.hono.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='red',marker_size=20,line_width=3,name="HONO"))
fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='red',marker_size=20,line_width=3,name="",showlegend=False))

x_data=flux_df.nitrate_molecules_density.values
y_data=flux_df.no2.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='orange',marker_size=20,line_width=3,name="NO<sub>2</sub> "))
fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='orange',marker_size=20,line_width=3,name="",showlegend=False))


x_data=flux_df.nitrate_molecules_density.values
y_data=flux_df.no.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='navy',marker_size=20,line_width=3,name="NO "))
fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='navy',marker_size=20,line_width=3,name="",showlegend=False))

x_data=flux_df.nitrate_molecules_density.values
y_data=flux_df.total.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='blue',marker_size=20,line_width=3,name="total"))
fig_1.add_trace(go.Scatter(x=x_data*1e-20,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='blue',marker_size=20,line_width=3,name="",showlegend=False))


name='Nitrate_dependence'

fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="NH<sub>4</sub>NO<sub>3</sub> amount (10<sup>20</sup> molecules cm<sup>-2</sup>)",titlefont=dict(color="black",size=40),ticklen=6,tickwidth=3,  
               showline=True, mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside'),
    yaxis=dict( title='flux (10<sup>11</sup> molecules cm<sup>-2</sup> s<sup>-1</sup>)', zeroline=False,
              #  range=[0,1.5],
               titlefont=dict(color="black",size=40),mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside',ticklen=6,tickwidth=3,
               ),
    yaxis2=dict(title='',titlefont=dict(color="black",size=40),tickfont=dict(color="black" ,size=40),ticks='inside',
                anchor="free",overlaying="y", side="right", position=1, ticklen=0,tickwidth=3,linecolor="black", linewidth=2,showticklabels = False) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.95 ,xanchor="left",x=0.05,  font=dict( size=40,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
fig_1.show()
fig_1.write_image('output/'+name+'_gradient.svg')

y= 0.0000x + 32638465389.48
R2 = 0.5206
y= 0.0000x + 25603769308.70
R2 = 0.2804
y= 0.0000x + 5885971527.89
R2 = 0.3166
y= 0.0000x + 64128206226.08
R2 = 0.4896


In [90]:
e_aim_output=pd.read_csv('output/E_AIM_batch_nitrate_gradient.csv')
e_aim_output['solution_volume_list']


0    0.96497
1    4.82487
2    4.82487
3    6.75481
4    9.64973
5   19.29950
6   19.29950
7   19.29950
8   48.24870
9   96.49730
Name: solution_volume_list, dtype: float64

In [91]:

flux_df = {}
flux_df["nitrate_M"]=data_compile["nitrate_M"]
flux_df["nitrate_molecules_density"]=data_compile["nitrate_molecules_density"]
flux_df["DHB_M"]=data_compile["DHB_M"]
flux_df["DHB_nitrate_ratio"]=data_compile["DHB_nitrate_ratio"]

for column in ["hono", "no", "no2","total"]:
    flux_df[column] = data_compile[column]*data_compile["nitrate_M"]*volume_solution*constants.N_A/surface_area
flux_df=pd.DataFrame(flux_df)
flux_df
# data_compile.index=[1,0.5,0.2,0.1]

fig_1= go.Figure()
from import_ import linear

x_data=e_aim_output.solution_volume_list.values
y_data=flux_df.hono.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='red',marker_size=20,line_width=3,name="HONO"))
fig_1.add_trace(go.Scatter(x=x_data,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='red',marker_size=20,line_width=3,name="",showlegend=False))

x_data=e_aim_output.solution_volume_list.values
y_data=flux_df.no2.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='orange',marker_size=20,line_width=3,name="NO<sub>2</sub> "))
fig_1.add_trace(go.Scatter(x=x_data,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='orange',marker_size=20,line_width=3,name="",showlegend=False))


x_data=e_aim_output.solution_volume_list.values
y_data=flux_df.no.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='navy',marker_size=20,line_width=3,name="NO "))
fig_1.add_trace(go.Scatter(x=x_data,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='navy',marker_size=20,line_width=3,name="",showlegend=False))

x_data=e_aim_output.solution_volume_list.values
y_data=flux_df.total.values
y_predict=linear(x_data,y_data)

fig_1.add_trace(go.Scatter(x=x_data,y=y_data*1e-11,yaxis = "y1",mode='markers',marker_color='blue',marker_size=20,line_width=3,name="total"))
fig_1.add_trace(go.Scatter(x=x_data,y=y_predict.flatten()*1e-11,yaxis = "y1",mode='lines',marker_color='blue',marker_size=20,line_width=3,name="",showlegend=False))


name='Nitrate_dependence_2'
fig_1.update_layout(title='NH<sub>4</sub>NO<sub>3</sub> ('+"%.2f" % (e_aim_output.nh4_output_molarity_list[0])+' mol L<sup>-1</sup>)',titlefont=dict(color="black",size=25))

fig_1.update_layout(
  plot_bgcolor = 'white',paper_bgcolor = 'white',autosize=False,width=1000, height=800,
    xaxis=dict( title="Sample volume (uL)",titlefont=dict(color="black",size=40),ticklen=6,tickwidth=3,  
               showline=True, mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside'),
    yaxis=dict( title='flux (10<sup>11</sup> molecules cm<sup>-2</sup> s<sup>-1</sup>)', zeroline=False,
              #  range=[0,1.5],
               titlefont=dict(color="black",size=40),mirror= True,tickfont=dict(color='black',size=40 ),linecolor="black", linewidth=4.5,ticks='inside',ticklen=6,tickwidth=3,
               ),
    yaxis2=dict(title='',titlefont=dict(color="black",size=40),tickfont=dict(color="black" ,size=40),ticks='inside',
                anchor="free",overlaying="y", side="right", position=1, ticklen=0,tickwidth=3,linecolor="black", linewidth=2,showticklabels = False) )
fig_1.update_layout(legend=dict(yanchor="top",y=0.95 ,xanchor="left",x=0.05,  font=dict( size=40,color="black")))
fig_1.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
)
fig_1.show()
fig_1.write_image('output/'+name+'_gradient.svg')
fig_1.write_html(name+'_gradient.html')


y= 1166336776.6910x + 32638435825.70
R2 = 0.5206
y= 324066449.4470x + 25603758387.23
R2 = 0.2804
y= 307520209.1080x + 5885961272.11
R2 = 0.3166
y= 1797923435.2459x + 64128155485.04
R2 = 0.4896
